# Synopsis-to-Vector

In [1]:
import numpy as np
import pandas as pd
import pickle
import math

## Load Pre-trained CBOW Parameters

In [2]:
pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

## Get TF-IDF from synopsis.csv

### Load Cleaned Synopsis Data

In [3]:
synopsis = pd.read_csv('../clean_data/synopsis.csv')
synopsis.head()

,MAL_ID,Synopsis
0,1,solar leaving surface planet earth solar polic...
1,5,day life crew routine interrupted chasing targ...
2,6,head reason waste oppose entire cities fun tit...
3,7,individuals powers mind control robin craft us...
4,8,dark century people suffering rule manipulate ...


### Get Word Set from Synopsis Data

In [4]:
syns = synopsis.Synopsis.str.findall('\w+')

vocab = set()
for syn in syns:
    vocab.update(syn)

vocab = sorted(list(vocab))
vocab_size = len(vocab)

### Get TF(Term Frequency)

In [5]:
def get_tf(word, syn):
    # Term Frequency
    return syn.count(word)

    
tf = []
for syn in syns:
    tf_syn = []
    for word in vocab:
        tf_syn.append(get_tf(word, syn))
    tf.append(tf_syn)

tf = np.asarray(tf)
tf.shape

(6000, 5921)

### Get IDF(Inverse Document Frequency)

In [6]:
def get_idf(term):
    # Inverse Document Frequency
    _df = 0
    for syn in syns:
        _df += int(word in syn)
    return math.log(vocab_size/(_df))


idf = []
for word in vocab:
    idf.append(get_idf(word))

idf = np.asarray(idf)
idf.shape

(5921,)

### Get TF-IDF Table

In [7]:
tf_idf = pd.DataFrame(tf * idf, columns=vocab)

## Generate Sentence Vector of Synopsis

In [8]:
def get_word_vec(word):
    return word_vecs[word_to_id[word]]

def get_tf_idf(word, i):
    return tf_idf.loc[i, word]

def get_sent_vec(i):
    sent_vec = np.zeros(100, dtype=np.float16)

    for word in set(syns[i]):
        sent_vec += get_word_vec(word) * get_tf_idf(word, i)
    
    return sent_vec / len(set(syns[i]))

In [9]:
full_sent_vec = []
for i in range(len(syns)):
    full_sent_vec.append(get_sent_vec(i))
    
sent_vec_df = pd.DataFrame(full_sent_vec)
sent_vec_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.115601,-3.925781,0.509766,1.327148,2.687500,1.792969,-1.769531,-1.252930,-1.898438,1.072266,...,0.862305,-0.738770,-1.119141,-0.729004,1.063477,1.273438,-0.246826,1.380859,1.921875,2.218750
1,2.310547,-2.365234,-0.012993,2.703125,1.200195,3.167969,-1.358398,-2.605469,-1.960938,-0.566406,...,0.494141,-0.946289,-0.938965,-0.260742,2.087891,1.499023,-0.222168,0.645996,2.220703,0.959473
2,-0.225830,-2.601562,-0.183716,2.042969,1.671875,1.954102,-1.366211,-2.914062,-0.161499,0.051636,...,-1.436523,-1.020508,-2.537109,-2.455078,2.015625,1.945312,0.623047,1.704102,2.359375,1.232422
3,0.845703,-0.527344,0.271729,0.752930,1.575195,2.496094,-1.654297,-2.703125,-2.267578,1.470703,...,-0.368164,-1.790039,-1.804688,-2.478516,2.880859,0.426514,-0.583008,3.107422,0.838867,1.043945
4,1.522461,-2.654297,0.423584,1.816406,2.027344,1.694336,0.607422,-1.332031,-1.022461,0.937500,...,1.650391,-1.703125,-2.478516,-4.484375,1.400391,0.638672,0.058136,1.249023,2.455078,2.515625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.908691,-1.424805,1.901367,2.382812,1.666992,0.900879,-1.927734,-0.842773,-1.282227,1.338867,...,1.072266,-0.566406,-1.357422,-2.285156,0.418213,1.647461,0.828125,1.503906,2.166016,2.941406
5996,2.140625,-1.451172,0.351074,2.123047,0.429199,-0.171631,-0.984375,-2.195312,-2.027344,1.022461,...,-0.836426,-1.898438,-2.449219,-2.021484,1.229492,0.563477,-1.253906,2.990234,2.492188,1.021484
5997,1.147461,-2.382812,0.937988,0.431152,1.121094,2.529297,-1.701172,-1.219727,-1.711914,1.555664,...,0.712891,-0.081726,-1.974609,-0.666992,2.468750,0.973145,1.821289,-0.183350,2.945312,0.713867
5998,0.945801,-1.449219,-0.097412,0.824707,1.160156,1.100586,-0.375732,-2.132812,-0.812012,2.248047,...,1.913086,0.848633,-1.115234,-1.066406,1.033203,1.030273,-0.781250,0.202759,2.701172,1.934570


In [10]:
sent_vec_df.insert(0, 'MAL_ID', synopsis.MAL_ID)
sent_vec_df

,MAL_ID,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,1,-0.115601,-3.925781,0.509766,1.327148,2.687500,1.792969,-1.769531,-1.252930,-1.898438,...,0.862305,-0.738770,-1.119141,-0.729004,1.063477,1.273438,-0.246826,1.380859,1.921875,2.218750
1,5,2.310547,-2.365234,-0.012993,2.703125,1.200195,3.167969,-1.358398,-2.605469,-1.960938,...,0.494141,-0.946289,-0.938965,-0.260742,2.087891,1.499023,-0.222168,0.645996,2.220703,0.959473
2,6,-0.225830,-2.601562,-0.183716,2.042969,1.671875,1.954102,-1.366211,-2.914062,-0.161499,...,-1.436523,-1.020508,-2.537109,-2.455078,2.015625,1.945312,0.623047,1.704102,2.359375,1.232422
3,7,0.845703,-0.527344,0.271729,0.752930,1.575195,2.496094,-1.654297,-2.703125,-2.267578,...,-0.368164,-1.790039,-1.804688,-2.478516,2.880859,0.426514,-0.583008,3.107422,0.838867,1.043945
4,8,1.522461,-2.654297,0.423584,1.816406,2.027344,1.694336,0.607422,-1.332031,-1.022461,...,1.650391,-1.703125,-2.478516,-4.484375,1.400391,0.638672,0.058136,1.249023,2.455078,2.515625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,48438,0.908691,-1.424805,1.901367,2.382812,1.666992,0.900879,-1.927734,-0.842773,-1.282227,...,1.072266,-0.566406,-1.357422,-2.285156,0.418213,1.647461,0.828125,1.503906,2.166016,2.941406
5996,48466,2.140625,-1.451172,0.351074,2.123047,0.429199,-0.171631,-0.984375,-2.195312,-2.027344,...,-0.836426,-1.898438,-2.449219,-2.021484,1.229492,0.563477,-1.253906,2.990234,2.492188,1.021484
5997,48470,1.147461,-2.382812,0.937988,0.431152,1.121094,2.529297,-1.701172,-1.219727,-1.711914,...,0.712891,-0.081726,-1.974609,-0.666992,2.468750,0.973145,1.821289,-0.183350,2.945312,0.713867
5998,48483,0.945801,-1.449219,-0.097412,0.824707,1.160156,1.100586,-0.375732,-2.132812,-0.812012,...,1.913086,0.848633,-1.115234,-1.066406,1.033203,1.030273,-0.781250,0.202759,2.701172,1.934570


In [11]:
sent_vec_df.to_csv('synopsis_vector.csv')